In [1]:
# import the modules


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_auc_score ,mean_squared_error,accuracy_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import metrics
from IPython.display import Image
import pydotplus
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression

### Load the dataset

- Load the train data and using all your knowledge try to explore the different statistical properties of the dataset.

In [4]:
# Code starts here
data_train=pd.read_csv('/users/neha/Desktop/Pragra/Grey_Atom_Tasks/Sprint_8/Loan_Defaulter/train.csv')

data_train.info()

# Code ends here

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7662 entries, 0 to 7661
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Id                 7662 non-null   int64  
 1   customer.id        7662 non-null   object 
 2   credit.policy      7662 non-null   object 
 3   purpose            7662 non-null   object 
 4   int.rate           7662 non-null   object 
 5   installment        7662 non-null   float64
 6   log.annual.inc     7662 non-null   float64
 7   dti                7662 non-null   float64
 8   fico               7662 non-null   int64  
 9   days.with.cr.line  7662 non-null   float64
 10  revol.bal          7662 non-null   int64  
 11  revol.util         7662 non-null   float64
 12  pub.rec            7662 non-null   int64  
 13  inq.last.6mths     7662 non-null   object 
 14  delinq.2yrs        7662 non-null   object 
 15  paid.back.loan     7662 non-null   object 
dtypes: float64(5), int64(4),

In [ ]:
data_test=pd.read_csv('/users/neha/Desktop/Pragra/Grey_Atom_Tasks/Sprint_8/file/test.csv')

In [ ]:
data_train.head()

In [ ]:
data_train.describe()

In [ ]:
data_train['inq.last.6mths'].value_counts()

In [ ]:
data_train['purpose'].value_counts()

In [ ]:
data_train.loc[data_train['paid.back.loan']=='Yes', 'paid.back.loan'] = 1
data_train.loc[data_train['paid.back.loan']=='No', 'paid.back.loan'] = 0

In [ ]:
data_train['paid.back.loan'].value_counts()

### Visualize the data

- Check for the categorical & continuous features. 
- Check out the best plots for plotting between categorical target and continuous features and try making some inferences from these plots.
- Clean the data, apply some data preprocessing and engineering techniques.

In [ ]:
# Code starts here

# fig = plt.figure(figsize=(25,15))
# cols = 5
# row = np.ceil(float(data_train.shape[1])/cols)
# for i , column in enumerate(data_train.columns):
#     ax = fig.add_subplot(row,cols,i+1)
#     ax.set_title(column)
#     if data_train.dtypes[column] == np.object:
#         data_train[column].value_counts().plot(kind = 'bar',axes=ax)
#     else:
#         data_train[column].hist(axes=ax)
#         plt.xticks(rotation = 'vertical')
        
# plt.subplots_adjust(hspace=.7,wspace=.2)

# Code ends here.

In [ ]:
data_train.drop(['Id','customer.id'],1,inplace= True)

In [ ]:
categorical_columns = [c for c in data_train.columns 
                       if data_train[c].dtype.name == 'object']

for column in categorical_columns:
    data_train[column]=data_train[column].astype('category').cat.codes
    
print(data_train.info())

In [ ]:
X=data_train.drop(['paid.back.loan'], axis=1)
y=data_train['paid.back.loan'].copy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Model building

- Separate the features and target.
- Now let's come to the actual task, using Decision Tree, predict the `paid.back.loan`. Use different techniques you have learned to imporove the performance of the model.
- Try improving upon the `accuracy_score` ([Accuracy Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html))

In [ ]:
# Code Starts here
tree = DecisionTreeClassifier(max_depth=3, random_state=17)
tree.fit(X_train, y_train)
tree_predictions = tree.predict(X_test) 
print("Decision tree accuracy: ",accuracy_score(y_test,tree_predictions))
# Code ends here

In [ ]:
tree_params = {'max_depth': range(2, 11)}
locally_best_tree = GridSearchCV(DecisionTreeClassifier(random_state=17),tree_params, cv=5)                  
locally_best_tree.fit(X_train, y_train)

In [ ]:
print("Best params:", locally_best_tree.best_params_)
print("Best cross validaton score", locally_best_tree.best_score_)

In [ ]:
tuned_tree = DecisionTreeClassifier(max_depth=2, random_state=17)
tuned_tree.fit(X_train, y_train)
tuned_tree_predictions = tuned_tree.predict(X_test)
print("Decision tree Accuracy after tuning: ",accuracy_score(y_test, tuned_tree_predictions))

### Prediction on the test data and creating the sample submission file.

- Load the test data and store the `Id` column in a separate variable.
- Perform the same operations on the test data that you have performed on the train data.
- Create the submission file as a `csv` file consisting of the `Id` column from the test data and your prediction as the second column.

In [ ]:
# Code Starts here

data_test.drop(['customer.id'],1,inplace= True)

# Code ends here

In [ ]:
categorical_columns = [c for c in data_test.columns 
                       if data_test[c].dtype.name == 'object']

for column in categorical_columns:
    data_test[column]=data_test[column].astype('category').cat.codes
    
print(data_test.info())

In [ ]:

# Storing the id from the test file
id_ = data_test['Id']

# Dropping the same columns from the test data
data_test.drop(['Id'],1,inplace=True)

# Applying rfe on test data

y_pred_test= tuned_tree.predict(data_test)

# Predict on the test data

# Create a sample submission file
final_submission = pd.DataFrame({'Id':id_,'paid.back.loan':y_pred_test})

final_submission.loc[final_submission['paid.back.loan']==1, 'paid.back.loan'] = 'Yes'
final_submission.loc[final_submission['paid.back.loan']==0, 'paid.back.loan'] = 'No'

# Convert the sample submission file into a csv file
final_submission.to_csv('final_submission.csv',index=False)